In [1]:
using Distributions
using Random
using DataFrames
using CSVFiles
include("./polya.jl")
using .Polya

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1260
┌ Info: Precompiling CSVFiles [5d742f6a-9f54-50ce-8119-2520741973ca]
└ @ Base loading.jl:1260


In [2]:
function compute_KLD(share::Array{Float64,N}, p::Vector{Float64}) where {N}
    all(sum(share, dims=1) .≈ 1) || throw(ArgumentError("Shares have to sum to 1.")) 
    deviation = log.(share ./ p)
    # log(0) is fine because 0*log(0) = 0
    replace!(deviation, -Inf=>0)
    # convert to vector instead of 1xK array
    return sum(share .* deviation, dims=1)
end

compute_KLD (generic function with 1 method)

In [10]:
function compute_p_values(A)
    # exclude empty rows
    data = A[vec(sum(A, dims=2) .> 0), :]
    K, N = size(data)
    
    # if no data left, return p = 1.0 because we cannot reject the null
    K > 0 || return ones(Float64, N)
    
    # countries with no shipments at all
    empty_columns = vec(sum(data, dims=1) .== 0)
    # add 1 shipment to these so that algorithm completes but change p = 1 later
    data[1, empty_columns] .= 1
    
    H0_params = Polya.mle(DirichletMultinomial, data, tol=1e-4)
    H0_shares = H0_params.α ./ sum(H0_params.α)
    actual_KLD = compute_KLD(data ./ sum(data, dims=1), H0_shares)
    
    p = zeros(Float64, N)
    for i = 1:N
        # actual number of shipments treated as a parameter
        H1 = DirichletMultinomial(sum(data[:,i]), H0_params.α)
        pmf = Polya.simulate_ECDF(H1, 
            x -> compute_KLD(x ./ sum(x, dims=1), 
                    H0_params.α ./ H0_params.α0), 
            maxiter=10000, digits=3)
        p[i] = 1 - cdf(pmf, actual_KLD[1,i])
    end
    # we have no information to reject the null
    p[empty_columns] .= 1.0
    return p
end

compute_p_values (generic function with 1 method)

In [4]:
data = DataFrame(load("../temp/shipment-clean.csv"))

,iso2_o,iso2_d,year,shipments1,shipments2,shipments3,shipments4,shipments5
,String,String,Int64,Int64,Int64,Int64,Int64,Int64
1,AT,AD,2001,0,0,0,0,0
2,AT,AD,2002,0,0,0,0,0
3,AT,AD,2003,0,0,0,0,0
4,AT,AD,2004,0,0,0,0,0
5,AT,AD,2005,0,0,0,0,0
6,AT,AD,2006,2,0,0,0,0
7,AT,AD,2007,1,0,0,0,0
8,AT,AD,2008,2,0,0,0,0
9,AT,AD,2009,0,0,0,0,0


In [5]:
function get_destination_matrix(data; country::String, year::Int = 2017)
    return filter(row -> row.iso2_d == country && row.year == year, data)[:,4:end-1]
end

get_destination_matrix (generic function with 1 method)

In [6]:
function flip(df::DataFrame) :: Array
    return Array(Array(df)')
end

flip (generic function with 1 method)

In [11]:
years = unique(data.year)
destinations = unique(data.iso2_d)
ps = copy(data[1:0,1:3])
ps.p = zeros(Float64, size(ps, 1))
for d in destinations
    for t in years 
        println(d, t)
        subset = get_destination_matrix(data, country=d, year=t)
        p = compute_p_values(flip(subset))
        new_batch = filter(row -> row.iso2_d == d && row.year == t, data)[:,1:3]
        new_batch[:,:p] = p
        append!(ps, new_batch)
    end
end


AD2001
AD2002
AD2003
AD2004
AD2005
AD2006
AD2007
AD2008
AD2009
AD2010
AD2011
AD2012
AD2013
AD2014
AD2015
AD2016
AD2017
AE2001
AE2002
AE2003
AE2004
AE2005
AE2006
AE2007
AE2008
AE2009
AE2010
AE2011
AE2012
AE2013
AE2014
AE2015
AE2016
AE2017
AF2001
AF2002
AF2003
AF2004
AF2005
AF2006
AF2007
AF2008
AF2009
AF2010
AF2011
AF2012
AF2013
AF2014
AF2015
AF2016
AF2017
AG2001
AG2002
AG2003
AG2004
AG2005
AG2006
AG2007
AG2008
AG2009
AG2010
AG2011
AG2012
AG2013
AG2014
AG2015
AG2016
AG2017
AI2001
AI2002
AI2003
AI2004
AI2005
AI2006
AI2007
AI2008
AI2009
AI2010
AI2011
AI2012
AI2013
AI2014
AI2015
AI2016
AI2017
AL2001
AL2002
AL2003
AL2004
AL2005
AL2006
AL2007
AL2008
AL2009
AL2010
AL2011
AL2012
AL2013
AL2014
AL2015
AL2016
AL2017
AM2001
AM2002
AM2003
AM2004
AM2005
AM2006
AM2007
AM2008
AM2009
AM2010
AM2011
AM2012
AM2013
AM2014
AM2015
AM2016
AM2017
AN2001
AN2002
AN2003
AN2004
AN2005
AN2006
AN2007
AN2008
AN2009
AN2010
AN2011
AN2012
AN2013
AN2014
AN2015
AN2016
AN2017
AO2001
AO2002
AO2003
AO2004
AO2005
AO2006
AO2007

ArgumentError: ArgumentError: alpha must be a positive vector.

In [8]:
ps

,iso2_o,iso2_d,year,p
,String,String,Int64,Float64
1,AT,AD,2001,0.629563
2,BE,AD,2001,0.127313
3,BG,AD,2001,0.532553
4,CY,AD,2001,0.609861
5,CZ,AD,2001,0.90099
6,DE,AD,2001,0.20362
7,DK,AD,2001,0.00130013
8,EE,AD,2001,0.00430043
9,ES,AD,2001,1.0


In [9]:
save("../temp/p-values.csv", ps)